In [47]:
import pandas as pd

def classify_bars(df):
    """
    Classify each bar as:
    DB  = Directional Bar
    ISB = Inside Bar
    OSB = Outside Bar
    """
    df['bartype'] = None

    # First bar is always DB
    df.iloc[0, df.columns.get_loc('bartype')] = "DB"

    for i in range(1, len(df)):
        prev_high = df.iloc[i-1]['high']
        prev_low = df.iloc[i-1]['low']
        high = df.iloc[i]['high']
        low = df.iloc[i]['low']

        # ISB: inside previous bar
        if high < prev_high and low > prev_low:
            df.at[df.index[i], 'bartype'] = "ISB"

        # OSB: outside previous bar
        elif high > prev_high and low < prev_low:
            df.at[df.index[i], 'bartype'] = "OSB"

        # All other cases are DB (normal direction bar)
        else:
            df.at[df.index[i], 'bartype'] = "DB"
    return df

In [99]:
def generate_swings(df):
    df['swing'] = None

    for i in range(1, len(df)):
        high = df.iloc[i]['high']
        low = df.iloc[i]['low']
        bartype = df.iloc[i]['bartype']

        prev_high = df.iloc[i-1]['high']
        prev_low = df.iloc[i-1]['low']
        previous_swing = df.iloc[i-1]['swing']
        current_swing = None

        if bartype == "DB":
            if high > prev_high:
                df.at[df.index[i], 'swing'] = "high"
                current_swing = "high"

            elif low < prev_low:
                df.at[df.index[i], 'swing'] = "low"
                current_swing = "low"

        if bartype == "ISB":
            df.at[df.index[i], 'swing'] = previous_swing
            current_swing = previous_swing


    return df


In [103]:
df = pd.read_json("nifty-dataset.json")

df['datetime'] = pd.to_datetime(df['datetime'])
df = df[df["datetime"] >= pd.to_datetime("2025-06-25")]

df = classify_bars(df)
df = generate_swings(df)
print(df[['datetime', 'close', 'bartype', 'swing']])


      datetime    close bartype swing
119 2025-06-25  25252.2      DB  None
120 2025-06-26  25551.0      DB  high
121 2025-06-27  25750.2      DB  high
122 2025-06-30  25614.2     OSB  None
123 2025-07-01  25643.3     ISB  None
..         ...      ...     ...   ...
216 2025-11-14  25951.4      DB   low
217 2025-11-17  26060.1      DB  high
218 2025-11-18  25948.8      DB   low
219 2025-11-19  26071.0     OSB  None
220 2025-11-20  26235.5      DB  high

[102 rows x 4 columns]
